# Create the features for the ActionFormerObj Code

To use this code you should have done the first extraction, in our case we extracted 16-frames features with X-CLIP

## 1. Imports

In [2]:
import numpy as np
import os
import glob
import json
import math
import torch
import pandas as pd

## 2. Util functions

In [3]:
def load_npy_file(file_path):
    return np.load(file_path)

def load_json_file(json_path):
    with open(json_path, 'r') as f:
        return json.load(f)

In [4]:
def combine_feat(movie_length_minutes,scene_length_minutes, fps, window, stride):
    """
    Args:
        movie_length_minutes (str): Movie length in minutes
        scene_length_minutes (_type_): Desired scene length in minutes
        fps (float): Movie frame rate per seconds
        window (int): Window sized used when extrating the features with X-CLIP
        stride (int): Stride used when extracting the features with X-CLIP

    Returns:
        list: list of features delimitations
    """

    scene_length_frames = scene_length_minutes * fps * 60
    movie_length_frames = movie_length_minutes * fps * 60


    start_scene = np.arange(0, movie_length_frames - 1, scene_length_frames)
    end_scene = np.arange(scene_length_frames,movie_length_frames, scene_length_frames )


    start_cuts = np.repeat(np.arange(movie_length_frames // window + 1), window).astype(int)
    end_cuts = np.repeat(np.arange(movie_length_frames // stride + 1), stride).astype(int)


    list_feat = []
    for scene_num, (ss,es) in enumerate(zip(start_scene, end_scene)):
        
        ss_int = math.floor(ss)
        es_int = math.ceil(es)

        sfeat = start_cuts[ss_int]
        efeat = end_cuts[es_int]
        list_feat.append((sfeat, efeat))

    return list_feat



In [5]:
def group_feat(original_feat_dir, saving_feat_dir, scene_length_minutes, fps, window, stride):
    """Concatenate the feature to obtain one tensor storing all the features for a scene lasting scene_length_minutes.

    Args:
        original_feat_dir (str or pathlike): Path to the folder storing the features extracted from X-CLIP
        saving_feat_dir (str or pathlike): Path to the folder that will stores the new features
        scene_length_minutes (int): Desired scene length in minutes
        fps (float): Movie frame rate per seconds
        window (int): Window sized used when extrating the features with X-CLIP
        stride (int): Stride used when extracting the features with X-CLIP
    """

    file_names=os.listdir(original_feat_dir)
    imdb_id = file_names[0].split('_')[0]

    movie_length_minutes = (len(file_names) * stride - 1 ) / (fps * 60)
    
    delimitations = combine_feat(movie_length_minutes,scene_length_minutes, fps, window, stride)
    print("NB SCENES : ", len(delimitations))
    
    for scene_number, (start, end) in enumerate(delimitations):
        tensors2load = [torch.load(os.path.join(original_feat_dir,imdb_id + "_" + str(j) +".pt" ))for j in range(start,end+1) if os.path.exists(os.path.join(original_feat_dir,imdb_id + "_" + str(j) +".pt" ))]
        res = torch.cat(tensors2load, dim = 0)
        torch.save(res, os.path.join(saving_feat_dir,imdb_id + "_" + str(scene_number).zfill(3) +".pt" ))

## 3. Parameters 

In [ ]:
original_root_dir = "/media/LaCie/Features/Clip_16_Frames/" # Path to our original 16-frames features
saving_root_dir = "/media/LaCie/Features/Scene_5_minutes" # Path to our new folder storing the new features

annotation_path = "dataset/mobygaze_dataframe.csv" # Path to the dataframe storing the annotations
fps_df = pd.read_csv(annotation_path, sep=";", index_col=0).loc[:, ["imdb_key","framerate"]]


SCENE_LENGTH_MIN = 5 # Scene length in minutes
WINDOW = 16 # window size used to extract the features
STRIDE = 16 # stride used to extract the features


## 4. Process the features

In [ ]:
fps_df = fps_df.groupby(by="imdb_key").max()
imdb_ids = os.listdir(original_root_dir)

for imdb_id in imdb_ids:
    
    if not "tt" in imdb_id:continue
    print("IMDB :", imdb_id)
    fps = fps_df.loc[imdb_id].framerate
    original_feat_dir = os.path.join(original_root_dir, imdb_id)
    saving_feat_dir = os.path.join(saving_root_dir, imdb_id)
    if not os.path.exists(saving_feat_dir) : os.makedirs(saving_feat_dir)
    group_feat(original_feat_dir, saving_feat_dir, SCENE_LENGTH_MIN, fps, WINDOW, STRIDE)
    print("\n")